In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

"""
# Set random seed
seed = np.random.seed(42)

# Generate synthetic dataset
X, y = make_classification(
    n_samples=5000,
    n_features=9,
    n_informative=6,
    n_redundant=0,
    n_classes=2,
    random_state=seed
)

#DataFrame
columns = ['Age', 'Gender','Height', 'Weight', 'Vitamin_Deficiency', 'Dietary_Preferences',
           'Allergies', 'Activity_Level', 'Cause']
df = pd.DataFrame(X, columns=columns)
df['Health'] = y

df['Age'] = np.random.randint(18, 70, size=len(df))
df['Height'] = np.random.randint(150, 190, size=len(df))
df['Weight'] = np.random.randint(48, 90, size=len(df))


df['Gender'] = np.random.choice([0, 1], size=len(df))

# Mapping
vitamin_mapping = {'Vitamin_A': 0, 'Vitamin_B': 1, 'Vitamin_C': 2, 'Vitamin_D': 3}
diet_mapping = {'Vegetarian': 0, 'Vegan': 1, 'Omnivore': 2}
allergy_mapping = {'None': 0, 'Peanuts': 1, 'Dairy': 2, 'Shellfish': 3}
activity_mapping = {'Low': 0, 'Moderate': 1, 'High': 2}
cause_mapping = {'Genetic': 0, 'Environmental': 1, 'Lifestyle': 2}

#mapping to categorical columns
df['Vitamin_Deficiency'].fillna(df['Vitamin_Deficiency'].map(vitamin_mapping), inplace=True)
df['Dietary_Preferences'].fillna(df['Dietary_Preferences'].map(diet_mapping), inplace=True)
df['Allergies'].fillna(df['Allergies'].map(allergy_mapping), inplace=True)
df['Activity_Level'].fillna(df['Activity_Level'].map(activity_mapping), inplace=True)
df['Cause'].fillna(df['Cause'].map(cause_mapping), inplace=True)
print(df.head())
"""

In [3]:
# Step 1: Load the data
df = pd.read_csv("code.datasett.csv")
df.head()

,age,gender,height,weight,vitamin_def,dietary_pref,activity_level,allergies,health,cause,food_rec
0,53,1,191.824829,46.967988,iron,vegan,High,Gluten,Average,None,Fruit Smoothie
1,41,1,172.386444,93.483996,iron,Vegetarian,Low,None,Average,Heart Disease,greens
2,23,0,173.919911,103.988173,Vit C,Non-vegetarian,Moderate,Shellfish,Average,Diabetes,greens
3,25,1,170.169125,138.183581,Vit C,vegan,Moderate,None,Good,Heart Disease,Salad
4,23,0,168.659846,89.946989,Vit A,Vegetarian,Low,None,Good,None,Salad


In [33]:
X = df.drop(columns=['food_rec'],axis=1)

In [34]:
y = df['food_rec']

In [36]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns

In [35]:
categorical_features = X.select_dtypes(include=['object']).columns

In [37]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define preprocessing steps for numerical and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with mean
    ('scaler', StandardScaler())  # Standardize numerical features
])

categorical_features = X.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent value
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])

# Combine preprocessing steps for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [38]:
from sklearn.feature_selection import SelectKBest, f_classif

# Define feature engineering pipeline
feature_engineering_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=5)),  # Perform PCA for dimensionality reduction
    ('feature_selection', SelectKBest(score_func=f_classif, k=5))  # Select top k features using ANOVA F-value
])


In [45]:
X_fe = feature_engineering_pipeline.fit_transform(X, y)

In [48]:
# Perform one-hot encoding for the 'food_rec' column
encoded_data = pd.get_dummies(df, columns=['food_rec'], prefix='food')

# Print the encoded data
encoded_data.head()

,age,gender,height,weight,vitamin_def,dietary_pref,activity_level,allergies,health,cause,food_Fruit Smoothie,food_Grilled Chicken,food_Salad,food_Steamed Vegetables,food_greens
0,53,1,191.824829,46.967988,iron,vegan,High,Gluten,Average,None,1,0,0,0,0
1,41,1,172.386444,93.483996,iron,Vegetarian,Low,None,Average,Heart Disease,0,0,0,0,1
2,23,0,173.919911,103.988173,Vit C,Non-vegetarian,Moderate,Shellfish,Average,Diabetes,0,0,0,0,1
3,25,1,170.169125,138.183581,Vit C,vegan,Moderate,None,Good,Heart Disease,0,0,1,0,0
4,23,0,168.659846,89.946989,Vit A,Vegetarian,Low,None,Good,None,0,0,1,0,0


In [61]:
X = X_fe
y = encoded_data.iloc[:, -4:]

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### **Splitting the data into train and test**

In [ ]:
"""
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
"""

In [ ]:
# PCA for dimensionality reduction
pca = PCA(n_components=0.95)  # Retain 95% of variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

**===============================================**

### **Random Forest Classifier**

In [ ]:
# Random Forest Classifier with Hyperparameter Tuning with PCA
param_grid_rf_pca = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_classifier_pca = RandomForestClassifier(random_state=42)
grid_search_rf_pca = GridSearchCV(estimator=rf_classifier_pca, param_grid=param_grid_rf_pca, cv=5)
grid_search_rf_pca.fit(X_train_pca, y_train)
best_rf_classifier_pca = grid_search_rf_pca.best_estimator_

# Model Evaluation
y_train_pred_rf_pca = best_rf_classifier_pca.predict(X_train_pca)
train_accuracy_rf_pca = accuracy_score(y_train, y_train_pred_rf_pca)

y_test_pred_rf_pca = best_rf_classifier_pca.predict(X_test_pca)
test_accuracy_rf_pca = accuracy_score(y_test, y_test_pred_rf_pca)

print('Training Accuracy:', train_accuracy_rf_pca)
print('Testing Accuracy:', test_accuracy_rf_pca)
print('-' * 30)

# Other Evaluation Metrics
print('Classification Report:')
print(classification_report(y_test, y_test_pred_rf_pca))

Training Accuracy: 1.0
Testing Accuracy: 0.917
------------------------------
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.92      0.92       489
           1       0.92      0.91      0.92       511

    accuracy                           0.92      1000
   macro avg       0.92      0.92      0.92      1000
weighted avg       0.92      0.92      0.92      1000



**The above model is built with PCA data. We are not going ahead with the above model because the model is giving the same results as the model without PCA.**


**Now, we knew that the model is performing the same even after applying Principal Component Analysis(PCA), there is no point in applying the technique for other models. So we are proceeding with the scaled data only!**

In [ ]:
# Random Forest Classifier with Hyperparameter Tuning
param_grid_rf2 = {
    'n_estimators': [20, 30, 40],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_classifier2 = RandomForestClassifier(random_state=42)
grid_search_rf2 = GridSearchCV(estimator=rf_classifier2, param_grid=param_grid_rf2, cv=10)
grid_search_rf2.fit(X_train, y_train)
best_rf_classifier2 = grid_search_rf2.best_estimator_

# Model Evaluation
y_train_pred_rf2 = best_rf_classifier2.predict(X_train)
train_accuracy_rf2 = accuracy_score(y_train, y_train_pred_rf2)

y_test_pred_rf2 = best_rf_classifier2.predict(X_test)
test_accuracy_rf2 = accuracy_score(y_test, y_test_pred_rf2)

print('Training Accuracy:', train_accuracy_rf2)
print('Testing Accuracy:', test_accuracy_rf2)
print('-' * 30)

# Other Evaluation Metrics
print('Classification Report:')
print(classification_report(y_test, y_test_pred_rf2))

Training Accuracy: 0.91675
Testing Accuracy: 0.879
------------------------------
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.84      0.87       489
           1       0.86      0.91      0.89       511

    accuracy                           0.88      1000
   macro avg       0.88      0.88      0.88      1000
weighted avg       0.88      0.88      0.88      1000



In [ ]:
# Updated parameter grid with further adjustments

param_grid_rf1 = {
    'n_estimators': [50, 100],    # Number of trees in random forest
    'max_depth': [None, 5],       # Maximum number of levels in tree
    'min_samples_split': [5, 10], # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2]    # Minimum number of samples required at each leaf node
}


rf_classifier1 = RandomForestClassifier(random_state=42)
grid_search_rf1 = GridSearchCV(estimator=rf_classifier1, param_grid=param_grid_rf1, cv=5) #cv=5 means our model will be executed 5 times (as CV given) and each time it will change Test data.
grid_search_rf1.fit(X_train, y_train)
best_rf_classifier1 = grid_search_rf1.best_estimator_


# Model Evaluation
y_train_pred_rf1 = best_rf_classifier1.predict(X_train)
train_accuracy_rf1 = accuracy_score(y_train, y_train_pred_rf1)

y_test_pred_rf1 = best_rf_classifier1.predict(X_test)
test_accuracy_rf1 = accuracy_score(y_test, y_test_pred_rf1)

print('Training Accuracy:', train_accuracy_rf1)
print('Testing Accuracy:', test_accuracy_rf1)
print('-' * 30)

# Other Evaluation Metrics
print('Classification Report:')
print(classification_report(y_test, y_test_pred_rf1))

Training Accuracy: 0.99475
Testing Accuracy: 0.903
------------------------------
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.89      0.90       489
           1       0.90      0.92      0.91       511

    accuracy                           0.90      1000
   macro avg       0.90      0.90      0.90      1000
weighted avg       0.90      0.90      0.90      1000





*   **As we can see in the above steps, we have tried changing the parameters
multiple times in order to increase the accuracies. Firstly, we tried with a complex model and started to reduce the complexity of model.**


*  **It is evident that the most complex model and the least complex model are giving almost the same accuracies infact the least complex model(the last one) is doing it in lessser time. So, we will keep that model for the comparison with other models and decide the best one to deploy!**



*   **We can also note that as we increase the complexity of model, the testing accuracy is further depleting.**








## **XG Boost Classifier**

In [ ]:
# XGBoost Classifier with Hyperparameter Tuning

from xgboost import XGBClassifier
param_grid_xgb = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.01]
}

xgb_classifier = XGBClassifier(random_state=42)
grid_search_xgb = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid_xgb, cv=5)
grid_search_xgb.fit(X_train, y_train)
best_xgb_classifier = grid_search_xgb.best_estimator_

y_train_pred = best_xgb_classifier.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

y_test_pred = best_xgb_classifier.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Training Accuracy:', train_accuracy)
print('Testing Accuracy:', test_accuracy)

### **KNN Classifier**

In [ ]:
# K-Nearest Neighbors Classifier with Hyperparameter Tuning

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

knn_classifier = KNeighborsClassifier()
grid_search_knn = GridSearchCV(estimator=knn_classifier, param_grid=param_grid_knn, cv=5)
grid_search_knn.fit(X_train_scaled, y_train)
best_knn_classifier = grid_search_knn.best_estimator_

"""
knn_classifier.fit(X_train_scaled, y_train)
prediction=knn_classifier.predict(X_test)

# Model Evaluation
y_train_pred = knn_classifier.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

y_test_pred = knn_classifier.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Training Accuracy:', train_accuracy)
print('Testing Accuracy:', test_accuracy)
print('-' * 30)
"""

"\nknn_classifier.fit(X_train_scaled, y_train)\nprediction=knn_classifier.predict(X_test)\n\n# Model Evaluation\ny_train_pred = knn_classifier.predict(X_train)\ntrain_accuracy = accuracy_score(y_train, y_train_pred)\n\ny_test_pred = knn_classifier.predict(X_test)\ntest_accuracy = accuracy_score(y_test, y_test_pred)\n\nprint('Training Accuracy:', train_accuracy)\nprint('Testing Accuracy:', test_accuracy)\nprint('-' * 30)\n"

### **Logistic Classifier**

In [ ]:
# Logistic Regression
logreg = LogisticRegression(max_iter=1000)
param_grid_logreg = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search_logreg = GridSearchCV(estimator=logreg, param_grid=param_grid_logreg, cv=5)
grid_search_logreg.fit(X_train_scaled, y_train)
best_logreg = grid_search_logreg.best_estimator_


### **Support Vector Machine**

In [ ]:
# Support Vector Machine (SVM)
svm = SVC(kernel='linear')
param_grid_svm = {'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto']}
grid_search_svm = GridSearchCV(estimator=svm, param_grid=param_grid_svm, cv=5)
grid_search_svm.fit(X_train_scaled, y_train)
best_svm = grid_search_svm.best_estimator_
#grid_search_svm.best_params_

### **Decision Tree Classifier**

In [ ]:
# Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()
param_grid_dt = {'max_depth': [None, 5, 10, 15, 20], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
grid_search_dt = GridSearchCV(estimator=dt_classifier, param_grid=param_grid_dt, cv=5)
grid_search_dt.fit(X_train_scaled, y_train)
best_dt_classifier = grid_search_dt.best_estimator_

### **Neural Network**

In [ ]:
#Neural Network

import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# Define the neural network model
nn_model = Sequential([
    Dense(16, input_dim=X_train.shape[1], activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)


# Model Evaluation
y_train_pred_nn = nn_model.predict(X_train)
y_test_pred_nn = nn_model.predict(X_test)

# Evaluate the model
train_accuracy = nn_model.evaluate(X_train, y_train, verbose=0)[1]
test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)[1]

# Model Evaluation
y_train_pred_nn = nn_model.predict(X_train)
y_test_pred_nn = nn_model.predict(X_test)


print('Training Accuracy:', train_accuracy)
print('Testing Accuracy:', test_accuracy)
print('-' * 30)


* **Although we got a good set of training and test accuracies with Random Forest Classifier, we have a significant difference between the train and test accuracies which is a potential sign of overfitting**.


* **But if we look at Neural Network model, the overall accuracy is good and the difference between train and test accuracies is comparatively less. So, this is minimizing the overfitting issue and the model is able to learn well.**

**METRICS**

In [ ]:

# Models Evaluation
models = {
    'Logistic Regression': best_logreg,
    'Support Vector Machine': best_svm,
    'Decision Tree Classifier': best_dt_classifier,
    'Random Forest Classifier1': best_rf_classifier1,
    'Random Forest Classifier2': best_rf_classifier2,
    'K Neighbors Classifier': best_knn_classifier,
    'XG Boost Classifier': best_xgb_classifier
}

for name, model in models.items():
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{name} accuracy:', accuracy)
    print(classification_report(y_test, y_pred))
    print('=' * 65)

Logistic Regression accuracy: 0.78
              precision    recall  f1-score   support

           0       0.79      0.75      0.77       489
           1       0.77      0.81      0.79       511

    accuracy                           0.78      1000
   macro avg       0.78      0.78      0.78      1000
weighted avg       0.78      0.78      0.78      1000

Support Vector Machine accuracy: 0.791
              precision    recall  f1-score   support

           0       0.81      0.75      0.78       489
           1       0.77      0.83      0.80       511

    accuracy                           0.79      1000
   macro avg       0.79      0.79      0.79      1000
weighted avg       0.79      0.79      0.79      1000

Decision Tree Classifier accuracy: 0.856
              precision    recall  f1-score   support

           0       0.85      0.85      0.85       489
           1       0.86      0.86      0.86       511

    accuracy                           0.86      1000
   macro avg 

In [ ]:
# Evaluate the Neural Network model
train_accuracy = nn_model.evaluate(X_train_scaled, y_train, verbose=0)[1]
test_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=0)[1]

print('Training Accuracy:', train_accuracy)
print('Testing Accuracy:', test_accuracy)

Training Accuracy: 0.9627500176429749
Testing Accuracy: 0.9459999799728394
------------------------------


In [ ]:
# Training and testing accuracies
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    train_accuracy = accuracy_score(y_train, model.predict(X_train_scaled))
    test_accuracy = accuracy_score(y_test, model.predict(X_test_scaled))

    print(f'{name}:')
    print(f'Training Accuracy: {train_accuracy:.4f}')
    print(f'Testing Accuracy: {test_accuracy:.4f}')
    print('-' * 30)

Logistic Regression:
Training Accuracy: 0.7915
Testing Accuracy: 0.7800
------------------------------
Support Vector Machine:
Training Accuracy: 0.7960
Testing Accuracy: 0.7910
------------------------------
Decision Tree Classifier:
Training Accuracy: 0.9425
Testing Accuracy: 0.8600
------------------------------
Random Forest Classifier1:
Training Accuracy: 0.9965
Testing Accuracy: 0.9070
------------------------------
Random Forest Classifier2:
Training Accuracy: 0.9167
Testing Accuracy: 0.8790
------------------------------
K Neighbors Classifier:
Training Accuracy: 1.0000
Testing Accuracy: 0.8960
------------------------------
XG Boost Classifier:
Training Accuracy: 0.9710
Testing Accuracy: 0.9170
------------------------------
